# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
1,taolanaro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bredasdorp,-34.5322,20.0403,48.27,89.0,8.0,6.76,ZA,1.627586e+09
3,mataura,-46.1927,168.8643,40.48,94.0,87.0,3.89,NZ,1.627586e+09
4,arraial do cabo,-22.9661,-42.0278,64.26,48.0,75.0,23.02,BR,1.627586e+09
...,...,...,...,...,...,...,...,...,...
606,inta,66.0317,60.1659,49.03,80.0,98.0,6.17,RU,1.627586e+09
607,songjianghe,42.1859,127.4790,67.53,97.0,100.0,3.83,CN,1.627586e+09
608,saint-joseph,-21.3667,55.6167,68.38,82.0,43.0,17.31,RE,1.627586e+09
609,cocorit,27.5833,-109.9667,95.34,35.0,21.0,10.42,MX,1.627586e+09


In [19]:
cities = cities.dropna()
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
2,bredasdorp,-34.5322,20.0403,48.27,89.0,8.0,6.76,ZA,1.627586e+09
3,mataura,-46.1927,168.8643,40.48,94.0,87.0,3.89,NZ,1.627586e+09
4,arraial do cabo,-22.9661,-42.0278,64.26,48.0,75.0,23.02,BR,1.627586e+09
5,geraldton,-28.7667,114.6000,65.12,82.0,75.0,20.71,AU,1.627586e+09
...,...,...,...,...,...,...,...,...,...
606,inta,66.0317,60.1659,49.03,80.0,98.0,6.17,RU,1.627586e+09
607,songjianghe,42.1859,127.4790,67.53,97.0,100.0,3.83,CN,1.627586e+09
608,saint-joseph,-21.3667,55.6167,68.38,82.0,43.0,17.31,RE,1.627586e+09
609,cocorit,27.5833,-109.9667,95.34,35.0,21.0,10.42,MX,1.627586e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
humidity = cities["Humidity"]
max_humidity = humidity.max()

In [21]:
locations = cities[["Lat","Lng"]]

###  Please note the Humidity Heat Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [22]:
# Plot Heatmap
fig = gmaps.figure()

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [24]:
ideal_cities = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) 
                          & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0)]
ideal_cities.head()                                                                                                                

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


In [25]:
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


In [26]:
ideal_cities.reset_index(inplace=True)
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09
1,94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09
2,104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09
3,168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09
4,192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_list = []

In [28]:
len(ideal_cities)

20

In [29]:
for index, row in ideal_cities.iterrows():
    lat = ideal_cities.loc[index]["Lat"]
    lng = ideal_cities.loc[index]["Lng"]
    
    # Use nearby search with a radius of 5000 for hotels
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
          "location": f'{lat},{lng}',
          "radius": 5000,
          "types": "hotel",
          "key": g_key
         }  
    
    requested = requests.get(base_url, params=params)
    hotel_jsn = requested.json()
    print(json.dumps(hotel_jsn, indent=4, sort_keys=True))
    try:
          hotel_list.append(hotel_jsn['results'][0]['name'])
    except:
          hotel_list.append("")
                
ideal_cities["Hotel Name"] = hotel_list
ideal_cities = ideal_cities.dropna()


{
    "html_attributions": [],
    "next_page_token": "Aap_uED2U34DIdrNMK-g0mZdg5b2HT0Dtsau2wq3WdZ7qLrwCb9QuPdMh8e81EXAK86ctmy5pVp2RAfZk2cAX3LdD_5nfNhcTEFNnDQ53Fev8lnwi9fd3N1woleF3T3GPoS0pyi8xar7a4U4D1bm2FLBHJA2LLFVcSWAsEkb7urHywWzlVWWsIK76ve1EcUiBgHOaQASZeNhxo5Na3gWcBmnTrZQInAGu393VLd_cHLrfOiG7lU_zlgq-jcFvA2w3Y9_LLhwq9meKOKt6ccSdc_gkHr0NURHZCVSJ0hbfId2H0n4WZ0A5EknrLDPlcjwoFdNWC4xU9-TeL_IL3qO9C32_m_IX8hFfthOiSXf78V7SMO7m_94HgHyM40-kmWg8Enrxk5kdC3BZXONDTZPV89bFnT9haeNkND4hj02qtW_W30l684CZ12hrw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 46.242273,
                    "lng": -60.23061339999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.26370508471251,
                        "lng": -60.20520694066628
                    },
                    "southwest": {
                        "lat": 46.21911197116021,
                        "lng": -60.264997

{
    "html_attributions": [],
    "next_page_token": "Aap_uECRxvwjvXMlrS-14A9Er7QGvmv83_HSLslnWAVM2dWqtct_UqdxDTjm2mtgrL-tR7pOEmmzjUY_WgCu569kPHATBx9PJCv8NGSYgcoKZL2qxDZprtIUxKgepvTdwvH0Abmr1ljOtjoodgwvUiA9PVbx9XSSroy_m9pu1Z1N-q5xsrDT_TFsXS60G9jY6FLr8kZyd5HTomHwnTVvTSJoOMqexUL9sJ7j3hysCn6do0yAlByHq7u6N_jSqX7vqmaqprllIQqs65rhHku-pzoyA4zHzymwUbjILvJ-JOIC7yjxzKyT67vBven7dmcIhuBftlRmuW9wQq4rXkqTOdtDHDxddY0gIS5XuZvuYhtN3bF0119Q6b8NaYCxlbTLGD8vPd-LzAxXmkgyIA5w0Nkt3y6fEDZpEIvMb_8oLVR5mKn8m8BlJBIwRQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 33.7241203,
                    "lng": 135.9924819
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.92213154686176,
                        "lng": 136.0122231338376
                    },
                    "southwest": {
                        "lat": 33.66953719406458,
                        "lng": 135.72237206471
 

{
    "html_attributions": [],
    "next_page_token": "Aap_uEALCZ2heknsUrWrP5W0cn971qX5p5SCIXct6i-PVw8nFmtJmPvhOw5_z1GLA2oySxZo42QLLgMMFE6vANLH30e3kWY6qGnBtZd3gPfMYB4bdzEbXCVERn1q0rzrtJT77GBGbT3xJoaqni6DtAKz7pS8VrRhLQAEYSexf33XjfHAPVA_OkavnA4qKSiJ5J0rVlpfgr8pNfiraivZwtSuvKK7lPQRddBIIX4VZjtP_YYI9yvtP7NXJEerSO5KfS_0ysDhkVYrvoF1Tdro_f_9f_kD8d4tj2TLKkZl8EcTukF8sfRFCJ51k2B27JTVmX0gQW8DcCO-2z9BD97jfcUUlylKgmjRpYMqGpG82-rtYrlPakXktiis_VApx8kURyHXaQq3K_j5iEGbQv2i5BpZQpvE7eODu_teO7O9FDA0o31tIxcptO9voA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 41.561959,
                    "lng": 35.905686
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.58482302323149,
                        "lng": 35.93230895823032
                    },
                    "southwest": {
                        "lat": 41.54361098003692,
                        "lng": 35.87890401661052
  

{
    "html_attributions": [],
    "next_page_token": "Aap_uEA5dUbgKt87W-TjvSAaBj01eJkNAPGfMkm9JtQEtuyiRbcbcTD4gsQTeyUrKUBM5oHGExZ1_RKMkAbmGFxtBTx17euu__4qLwca2ckfxqwbC1VL6urfg5DowAVaj0WOcvTvI05vFvlWIzXhG_3GK50HLqiy4PmgxjBIqe-ecBiy8ZYhurIkwuHdoMmnXveEjWnaNrdCE9aI1nzpGjIiXbCrTugoWwitQTgI_DkzcJa1z1HzzoLBjvqTk614ZZ6WnV7wsWxpNdPiRwR9FLlj5qvjGytyCgTbOv48wucTQujP1JArxqdhx-gvh5rLmK2ZqoLvRsJeYG-CfMpt1yRWsbd8qGPQR6m8nA6y_e7gTLvExFQe3EPNf_OKdAKMrovfFuUJfmQAYFgE4BcvdcBIru4qG2PltYz3tHBnwAYy2VBrRI06d7Z2CQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 47.5820518,
                    "lng": -53.2798524
                },
                "viewport": {
                    "northeast": {
                        "lat": 47.66875794604328,
                        "lng": -53.12573206069739
                    },
                    "southwest": {
                        "lat": 47.54990605891182,
                        "lng": -53.342792998735

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAZWGu9Pf84kFjzBmzVW2gLEcEhWrtjTWQoerb-voByuB84L9X3FsGrW2gS7o6flBvXE1doMD1TlR1_ipVdbH_PfMI1QPZTydijD0xHSO999lBkSj2oroiFkFmS02Jd96KkFoaG_LAca8G_PflXECNVk8oKjfNrFYQJsC9c7Jnf5Od9Wc3rW5TpIZXQ_407XyiaOMbQNEnYklT-FABByVfjh8xNtDajfDxGJvsfZVwQiJc8kUa6LWBv840TX9LIJ6C1cJT6UEmCe6yKAKlWDVYbCi69BJJIN3oEwC5EaoCmGv_J-MunBTMQXVM03naOwURoaANOfY4Kk8JPH0DSt9LfTxWzl0Y1Qo2oI_k5BNQ8wJeg2dZRTSnvNQl5uKrAtU-GTuNGKhSxzmtVZXzGIxUg6dQyOJXr1Cnxi2g8R_sOJIfCN_i1Uw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 39.772554,
                    "lng": 98.289419
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.8817444,
                        "lng": 98.406601
                    },
                    "southwest": {
                        "lat": 39.6890765,
                        "lng": 98.1326294
                    }
         

...

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAs-VPCjwCMvYDsSNftCxZJ0lWtM1EmJUNJUmljCS286fR3QJ0tDP15Y_mFlPmnsJysQmau-ekAKP4L2OUpkAPCf7GrBvE7zBEoZmjALEmMwVT0fSc46MKl_X4emT4aV0_j20-0zhohBhMlH27IleLsvAVSjtrsk7MbtKXe1MqKjgNxH243BlrJGnipVOVsK5yp7LR0onibH_Yqwg7QQAIg_knA631NUmItYVsbIzV6yysKCq3TsEWbOSdRQwi3-NvI6JHCBER_CYzCzPdIge5G6S_Yrnl4NaPzyvJ6wOH4ESC6OEuygUmsRqIf1eNx_gSxK9WbPois0xGQmQIZCJrshy-EPD7DUoSP_5kCtdGbPC1RjB7c2ycU5AIvvzFtnFE_4QIR9QIF4GtLklg0igjWd9f7L6VSulXv9k6HXzWJCHc-KuaFSw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -5.9127393,
                    "lng": 29.1913918
                },
                "viewport": {
                    "northeast": {
                        "lat": -5.876795360151529,
                        "lng": 29.24097065005702
                    },
                    "southwest": {
                        "lat": -5.965412246799739,
                        "lng": 29.1687011116837

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBhKFlgwVqfRWpFSBgfS4IfJ91jkLU23dCDTcHHKFZBSUnt1fnw_bIAOD41V2N-eNZah2ecBYKrNeGmxvxOb8C1F7DEcdiyLyFP4aYK6PJiGp2oBOQ2st4aEWdyrwbaklJdbJSaVTl99fx_j8ATolepVHYBwTfBkR0M7CRP1jd8C6vdI0n0wigzxejC9nfYF_WN3xpct42S8YcMKgjMPvxRs4a-mkXDyuWtaLcfv8An0DUdu723yWX4d_i91GaeLghrdmJsVF7izosOSx-ioeJ8UF9QU-6y7qmgS0HHE2hGTshmyuaGoTk_0nuCSXBWCAI3oL2Q-frbbv9RrVb0UacpMBLfOppNt3ZaxM7VQTreTMd2r0R0UHNVvrE-i4sTMBa-0GEow72YwTwm7Zg46vIhhuaonOdy2pBR4nbNTXJ42Tc7jg_Y8Q",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 38.1977107,
                    "lng": 67.2013657
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.23197775221099,
                        "lng": 67.24911689871183
                    },
                    "southwest": {
                        "lat": 38.15467201598199,
                        "lng": 67.16624740370422


{
    "html_attributions": [],
    "next_page_token": "Aap_uEA_li3imYWnovWo8QRi-XH-Cq6G8i7BpvYSnirBbCUneN4liA0GJEnZ_-chKTbbgrVIjFdKwQ5COzb0_Ylf62HhUkS1X8LcMi0H499TpICXixGobiEk2t_-0RTv6tEFrKmyevWbMfjgBxHLbSqbn_IQYJfshIxb23Ryr-74YTfRQf0MIhVpN8vCtEBI0fqgzNnyGF0Rij_Dq8bbqAkBc3HX8Enxsn7EdpIeJ-4pEOHGSaOl8sh1zup2DBmWZk9x3mJnBYeWbDi8Ymo2EAFpmAhQZkUI0n54KvpT_qS2TjXgkuS3PFd6RjDHeS_hwv23ydkzLZgckFFwY7KPzT6CyzrZMSHzdEAWH68iw1dww2GIEZf1ZKpDh7qUAQEjm093zrhA1yjpt5mlknHXS7q5PYeWREcyYoGLyjKWNPrFollWCVHX0z8tHQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 41.208647,
                    "lng": 36.972055
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.22104903904608,
                        "lng": 37.0044620125458
                    },
                    "southwest": {
                        "lat": 41.18983196410424,
                        "lng": 36.94897191866531
   

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAq1K1cKsfOwOEPFtbc-ZkxPT6zxEw7xEAER3JUncrIy-YdihlaurtmXwaZ-Izw9w1gILIZRCL99u9DPBtz-EJL8HDMeqU0teWNDzCf7urlI5csOKC3iXyVi7_SWemG8HnNrtnDLHZ-O5IdCYP9auwXrlc1iOpL0_IvrbwO8CWCXi0fJ4by9yXZ-o8Otke2zN2mjBxHBpGLsf5jT3N93hR7D7r1Ng930BSylxTXwHbAhkQZ5lpvDYWsf8JwDsN_tu1ERjKE1gw-1-jTMCXw5VHa9KMJmP6ru9q2L9VtJaR-HCOXvr4e7eL5hYS0NgV_LGtympkQTf-EiUFk_0sc7iRdTU1Ab7yS5BHFGXZ-tDyotzWEpgF8MkA9763KLa9hR9iQiFQc7Cb1zwcgUKv7yBPGcfIYWo9wObaNY1X87E38SELv0oWeZQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.5854485,
                    "lng": -70.69873609999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.52395396606174,
                        "lng": -70.6287438694405
                    },
                    "southwest": {
                        "lat": -33.75665475186723,
                        "lng": -70.828

{
    "html_attributions": [],
    "next_page_token": "Aap_uECBHJptLKisQPDwmkmSel28Fd52F_8XlTaOFhTUy66KstPFb98jDveQfOTv5VlII0dm9iXngIif2cbZOPQC4MPFp5Bm4eKQE_g1rrSg1X0DeTKIs_zj7PksLalLM8bpoQhToE9xyD8LosehdWpM0HNJJ7rfJE_LxDhNfNoau0Ovredy4yBT3lII14zhuaAQsEp2P-XdZjPQdhfCq23ihsDdD8Gi8DhQpctKl-pqpVG54uqQ_zZ2NQY2Z836o-FGBHnaO5E9eMd9J1cXblhDoGTdGU9K9EPES1Fb855F7MUhpXfEHhcT3Lh50c-vHoP-Ilj9pzGExbKlRsPQscQGlZwQlVrljGEkRofKU8ibJTkzP5ftyB_U_RJzmMiDQbrfQDvUrXXeB8Fr5fVK8aJqTWXMYbtfQFxTsMYO_RiWHm4cU_f-_0tW1Q",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 46.42347669999999,
                    "lng": 26.4258031
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.4396904935985,
                        "lng": 26.46344664520348
                    },
                    "southwest": {
                        "lat": 46.40652825894811,
                        "lng": 26.370448988

In [30]:
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,sydney mines,46.2367,-60.2177,73.09,51.0,0.0,5.99,CA,1.627586e+09,Sydney Mines
1,94,shingu,33.7333,135.9833,76.91,84.0,0.0,5.12,JP,1.627586e+09,Shingu
2,104,bafra,41.5678,35.9069,73.18,70.0,0.0,4.18,TR,1.627586e+09,Bafra
3,168,bay roberts,47.5999,-53.2648,73.65,54.0,0.0,8.99,CA,1.627586e+09,Bay Roberts
4,192,jiayuguan,39.8167,98.3000,76.23,19.0,0.0,3.83,CN,1.627586e+09,Jiayuguan City


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]


###  Please note the Hotel Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [32]:
# Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations)
locations = ideal_cities[["Lat", "Lng"]]
fig = gmaps.figure(center=(31.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

#fig = gmaps.figure()
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))